In [ ]:
# Allows user input for data set to label emotions
dataset_name = input('Dataset to label emotions (google, msr, quora, mix, twit0.825):  ')

if not (dataset_name.lower() in ['google', 'msr', 'quora', 'mix', 'twit0.825']): 
    print('Please enter a valid dataset name')

print('Dataset selected: ' + dataset_name)

training_stats = {'dataset name': dataset_name + '-training'}
eval_stats = {'dataset name': dataset_name + '-testing'}

In [ ]:
#Loading train_df and eval_df from files instead rerunning vader
import numpy as np
import pandas as pd
import os

train_df = pd.read_csv(f'compiled-data/{dataset_name}/{dataset_name}-training_emolabel_s.tsv', encoding='utf-8', sep="\t")
eval_df = pd.read_csv(f'compiled-data/{dataset_name}/{dataset_name}-testing_emolabel_s.tsv', encoding='utf-8', sep="\t")

In [ ]:
high_neg_emo = {'anger', 'disgust', 'grief', 'fear', 'sadness'}
low_neg_emo = {'nervousness', 'annoyance', 'disappointment', 'embarrassment', 'remorse', 'disapproval'}
neutral_emo = {'confusion', 'curiosity', 'realization', 'surprise', 'neutral'}
low_pos_emo = {'approval', 'caring', 'desire', 'relief'}
high_pos_emo = {'amusement', 'excitement', 'pride', 'optimism', 'gratitude', 'joy', 'admiration', 'love'}

high_neg_threshold = -0.4
high_pos_threshold = 0.45

In [ ]:
emotion_clusters = [ 
    ['anger', 'disgust', 'disapproval', 'annoyance'],
    ['love', 'joy', 'excitement', 'amusement'],
    ['desire', 'caring', 'optimism'],
    ['pride', 'admiration'],
    ['gratitude', 'relief'],
    ['approval', 'realization'],
    ['confusion', 'surprise', 'curiosity'],
    ['nervousness', 'fear', 'embarrassment', 'remorse'],
    ['disappointment', 'sadness', 'grief'],
    ['neutral']
]

emotion_transitions = {}

all_emotions = []
for cluster in emotion_clusters: 
    all_emotions += cluster
    for key in cluster: 
        if (key != 'neutral'):
            cluster_e = cluster + ['neutral']
            cluster_e.remove(key)
            emotion_transitions.update({key: cluster_e})

all_emotions.remove('neutral')
emotion_transitions.update({'neutral': all_emotions})

for item in emotion_transitions.items(): 
    print(item)

In [ ]:
sid_df = pd.DataFrame()
sid_rg_df = pd.DataFrame()
emo_df = pd.DataFrame()
emo_sid_df = pd.DataFrame()
emo_sid_tg_df = pd.DataFrame()
emo_sid_tg_nn_df = pd.DataFrame()
emo_sid_tg_ge_df = pd.DataFrame()
emo_sid_tg_nn_ge_df = pd.DataFrame()

sid_input_text, sid_target_text, sid_input_emo, sid_target_emo = [], [], [], []
sid_rg_input_text, sid_rg_target_text, sid_rg_input_emo, sid_rg_target_emo = [], [], [], []
emo_input_text, emo_target_text, emo_input_emo, emo_target_emo = [], [], [], []
emo_sid_input_text, emo_sid_target_text, emo_sid_input_emo, emo_sid_target_emo = [], [], [], []
emo_sid_tg_input_text, emo_sid_tg_target_text, emo_sid_tg_input_emo, emo_sid_tg_target_emo = [], [], [], []
emo_sid_tg_nn_input_text, emo_sid_tg_nn_target_text, emo_sid_tg_nn_input_emo, emo_sid_tg_nn_target_emo = [], [], [], []
emo_sid_tg_ge_input_text, emo_sid_tg_ge_target_text, emo_sid_tg_ge_input_emo, emo_sid_tg_ge_target_emo = [], [], [], []
emo_sid_tg_nn_ge_input_text, emo_sid_tg_nn_ge_target_text, emo_sid_tg_nn_ge_input_emo, emo_sid_tg_nn_ge_target_emo = [], [], [], []


for index, row in train_df.iterrows():
    if row.input_sid <= high_neg_threshold:
        input_sid_range = 'high_neg'
    elif row.input_sid > high_neg_threshold and row.input_sid < 0:
        input_sid_range = 'low_neg'
    elif row.input_sid == 0:
        input_sid_range = 'neutral'
    elif row.input_sid < high_pos_threshold and row.input_sid > 0:
        input_sid_range = 'low_pos'
    elif row.input_sid >= high_pos_threshold:
        input_sid_range = 'high_pos'
        
    if row.target_sid <= high_neg_threshold:
        target_sid_range = 'high_neg'
    elif row.target_sid > high_neg_threshold and row.input_sid < 0:
        target_sid_range = 'low_neg'
    elif row.target_sid == 0:
        target_sid_range = 'neutral'
    elif row.target_sid < high_pos_threshold and row.input_sid > 0:
        target_sid_range = 'low_pos'
    elif row.target_sid >= high_pos_threshold:
        target_sid_range = 'high_pos'
    
    if row.input_emo in high_neg_emo:
        input_emo_range = 'high_neg'
    elif row.input_emo in low_neg_emo:
        input_emo_range = 'low_neg'
    elif row.input_emo in neutral_emo:
        input_emo_range = 'neutral'
    elif row.input_emo in low_pos_emo:
        input_emo_range = 'low_pos'
    elif row.input_emo in high_pos_emo:
        input_emo_range = 'high_pos'
        
    if row.target_emo in high_neg_emo:
        target_emo_range = 'high_neg'
    elif row.target_emo in low_neg_emo:
        target_emo_range = 'low_neg'
    elif row.target_emo in neutral_emo:
        target_emo_range = 'neutral'
    elif row.target_emo in low_pos_emo:
        target_emo_range = 'low_pos'
    elif row.target_emo in high_pos_emo:
        target_emo_range = 'high_pos'
        
    if (row.input_sid < 0 and row.target_sid <= 0) & (row.input_sid < row.target_sid):
        sid_input_text.append(row.input_text)
        sid_target_text.append(row.target_text)
        sid_input_emo.append(row.input_sid)
        sid_target_emo.append(row.target_sid)
    
    elif (row.input_sid <= 0 and row.target_sid < 0) & (row.input_sid > row.target_sid):
        sid_input_text.append(row.target_text)
        sid_target_text.append(row.input_text)
        sid_input_emo.append(row.target_sid)
        sid_target_emo.append(row.input_sid)
    
    elif (row.input_sid > 0 and row.target_sid >= 0) & (row.input_sid > row.target_sid):
        sid_input_text.append(row.input_text)
        sid_target_text.append(row.target_text)
        sid_input_emo.append(row.input_sid)
        sid_target_emo.append(row.target_sid)
    
    elif (row.input_sid >= 0 and row.target_sid > 0) & (row.input_sid < row.target_sid):
        sid_input_text.append(row.target_text)
        sid_target_text.append(row.input_text)
        sid_input_emo.append(row.target_sid)
        sid_target_emo.append(row.input_sid)
        
        
    if ((input_sid_range == 'high_neg') and (target_sid_range == 'low_neg' or target_sid_range == 'neutral')) | \
       (input_sid_range == 'low_neg' and target_sid_range == 'neutral'):
        sid_rg_input_text.append(row.input_text)
        sid_rg_target_text.append(row.target_text)
        sid_rg_input_emo.append(input_sid_range)
        sid_rg_target_emo.append(target_sid_range)
    
    elif ((target_sid_range == 'high_neg') and (input_sid_range == 'low_neg' or input_sid_range == 'neutral')) | \
       (target_sid_range == 'low_neg' and input_sid_range == 'neutral'):
        sid_rg_input_text.append(row.target_text)
        sid_rg_target_text.append(row.input_text)
        sid_rg_input_emo.append(target_sid_range)
        sid_rg_target_emo.append(input_sid_range)
    
    elif ((input_sid_range == 'high_pos') and (target_sid_range == 'low_pos' or target_sid_range == 'neutral')) | \
       (input_sid_range == 'low_pos' and target_sid_range == 'neutral'):
        sid_rg_input_text.append(row.input_text)
        sid_rg_target_text.append(row.target_text)
        sid_rg_input_emo.append(input_sid_range)
        sid_rg_target_emo.append(target_sid_range)
    
    elif ((target_sid_range == 'high_pos') and (input_sid_range == 'pos' or input_sid_range == 'neutral')) | \
       (target_sid_range == 'low_pos' and input_sid_range == 'neutral'):
        sid_rg_input_text.append(row.target_text)
        sid_rg_target_text.append(row.input_text)
        sid_rg_input_emo.append(target_sid_range)
        sid_rg_target_emo.append(input_sid_range)
        
        
    if ((row.input_emo in high_neg_emo) & (row.target_emo in low_neg_emo.union(neutral_emo))) | \
       ((row.input_emo in low_neg_emo) & (row.target_emo in neutral_emo)): 
        emo_input_text.append(row.input_text)
        emo_target_text.append(row.target_text)
        emo_input_emo.append(input_emo_range)
        emo_target_emo.append(target_emo_range)
        
        if (row.input_sid < row.target_sid):
            emo_sid_input_text.append(row.input_text)
            emo_sid_target_text.append(row.target_text)
            emo_sid_input_emo.append(input_emo_range)
            emo_sid_target_emo.append(target_emo_range)
            
            if row.input_emo in emotion_transitions.keys() and row.target_emo in emotion_transitions[row.input_emo]:
                emo_sid_tg_input_text.append(row.input_text)
                emo_sid_tg_target_text.append(row.target_text)
                emo_sid_tg_input_emo.append(input_emo_range)
                emo_sid_tg_target_emo.append(target_emo_range)
                
                emo_sid_tg_ge_input_text.append(row.input_text)
                emo_sid_tg_ge_target_text.append(row.target_text)
                emo_sid_tg_ge_input_emo.append(row.input_emo)
                emo_sid_tg_ge_target_emo.append(row.target_emo)
                
                if ((row.input_emo != 'neutral') & (row.target_emo != 'neutral')):
                    emo_sid_tg_nn_input_text.append(row.input_text)
                    emo_sid_tg_nn_target_text.append(row.target_text)
                    emo_sid_tg_nn_input_emo.append(input_emo_range)
                    emo_sid_tg_nn_target_emo.append(target_emo_range)
                    
                    emo_sid_tg_nn_ge_input_text.append(row.input_text)
                    emo_sid_tg_nn_ge_target_text.append(row.target_text)
                    emo_sid_tg_nn_ge_input_emo.append(row.input_emo)
                    emo_sid_tg_nn_ge_target_emo.append(row.target_emo)

    elif ((row.input_emo in neutral_emo) & (row.target_emo in low_neg_emo.union(high_neg_emo))) | \
       ((row.input_emo in low_neg_emo) & (row.target_emo in high_neg_emo)):  
        emo_input_text.append(row.target_text)
        emo_target_text.append(row.input_text)
        emo_input_emo.append(target_emo_range)
        emo_target_emo.append(input_emo_range)
        
        if (row.input_sid > row.target_sid):
            emo_sid_input_text.append(row.target_text)
            emo_sid_target_text.append(row.input_text)
            emo_sid_input_emo.append(target_emo_range)
            emo_sid_target_emo.append(input_emo_range) 
            
            if row.input_emo in emotion_transitions.keys() and row.target_emo in emotion_transitions[row.input_emo]:
                emo_sid_tg_input_text.append(row.target_text)
                emo_sid_tg_target_text.append(row.input_text)
                emo_sid_tg_input_emo.append(target_emo_range)
                emo_sid_tg_target_emo.append(input_emo_range) 
                
                emo_sid_tg_ge_input_text.append(row.target_text)
                emo_sid_tg_ge_target_text.append(row.input_text)
                emo_sid_tg_ge_input_emo.append(row.target_emo)
                emo_sid_tg_ge_target_emo.append(row.input_emo)
                
                if ((row.input_emo != 'neutral') & (row.target_emo != 'neutral')):
                    emo_sid_tg_nn_input_text.append(row.target_text)
                    emo_sid_tg_nn_target_text.append(row.input_text)
                    emo_sid_tg_nn_input_emo.append(target_emo_range)
                    emo_sid_tg_nn_target_emo.append(input_emo_range) 
                    
                    emo_sid_tg_nn_ge_input_text.append(row.target_text)
                    emo_sid_tg_nn_ge_target_text.append(row.input_text)
                    emo_sid_tg_nn_ge_input_emo.append(row.target_emo)
                    emo_sid_tg_nn_ge_target_emo.append(row.input_emo)
    
    elif ((row.input_emo in high_pos_emo) & (row.target_emo in low_pos_emo.union(neutral_emo))) | \
       ((row.input_emo in low_pos_emo) & (row.target_emo in neutral_emo)): 
        emo_input_text.append(row.input_text)
        emo_target_text.append(row.target_text)
        emo_input_emo.append(input_emo_range)
        emo_target_emo.append(target_emo_range)
        
        if (row.input_sid > row.target_sid):
            emo_sid_input_text.append(row.input_text)
            emo_sid_target_text.append(row.target_text)
            emo_sid_input_emo.append(input_emo_range)
            emo_sid_target_emo.append(target_emo_range)
            
            if row.input_emo in emotion_transitions.keys() and row.target_emo in emotion_transitions[row.input_emo]:
                emo_sid_tg_input_text.append(row.input_text)
                emo_sid_tg_target_text.append(row.target_text)
                emo_sid_tg_input_emo.append(input_emo_range)
                emo_sid_tg_target_emo.append(target_emo_range)
                
                emo_sid_tg_ge_input_text.append(row.input_text)
                emo_sid_tg_ge_target_text.append(row.target_text)
                emo_sid_tg_ge_input_emo.append(row.input_emo)
                emo_sid_tg_ge_target_emo.append(row.target_emo)
                
                if ((row.input_emo != 'neutral') & (row.target_emo != 'neutral')):
                    emo_sid_tg_nn_input_text.append(row.input_text)
                    emo_sid_tg_nn_target_text.append(row.target_text)
                    emo_sid_tg_nn_input_emo.append(input_emo_range)
                    emo_sid_tg_nn_target_emo.append(target_emo_range)
                    
                    emo_sid_tg_nn_ge_input_text.append(row.input_text)
                    emo_sid_tg_nn_ge_target_text.append(row.target_text)
                    emo_sid_tg_nn_ge_input_emo.append(row.input_emo)
                    emo_sid_tg_nn_ge_target_emo.append(row.target_emo)
                    
    elif ((row.input_emo in neutral_emo) & (row.target_emo in low_pos_emo.union(high_pos_emo))) | \
       ((row.input_emo in low_pos_emo) & (row.target_emo in high_pos_emo)): 
        emo_input_text.append(row.target_text)
        emo_target_text.append(row.input_text)
        emo_input_emo.append(target_emo_range)
        emo_target_emo.append(input_emo_range)
        
        if (row.input_sid < row.target_sid):
            emo_sid_input_text.append(row.target_text)
            emo_sid_target_text.append(row.input_text)
            emo_sid_input_emo.append(target_emo_range)
            emo_sid_target_emo.append(input_emo_range) 
            
            if row.input_emo in emotion_transitions.keys() and row.target_emo in emotion_transitions[row.input_emo]:
                emo_sid_tg_input_text.append(row.target_text)
                emo_sid_tg_target_text.append(row.input_text)
                emo_sid_tg_input_emo.append(target_emo_range)
                emo_sid_tg_target_emo.append(input_emo_range) 
                
                if ((row.input_emo != 'neutral') & (row.target_emo != 'neutral')):
                    emo_sid_tg_nn_input_text.append(row.target_text)
                    emo_sid_tg_nn_target_text.append(row.input_text)
                    emo_sid_tg_nn_input_emo.append(target_emo_range)
                    emo_sid_tg_nn_target_emo.append(input_emo_range) 
                    
sid_df['input_text'], sid_df['target_text'], sid_df['input_emo'], sid_df['target_emo'] = sid_input_text, sid_target_text, sid_input_emo, sid_target_emo
sid_rg_df['input_text'], sid_rg_df['target_text'], sid_rg_df['input_emo'], sid_rg_df['target_emo'] = sid_rg_input_text, sid_rg_target_text, sid_rg_input_emo, sid_rg_target_emo
emo_df['input_text'], emo_df['target_text'], emo_df['input_emo'], emo_df['target_emo'] = emo_input_text, emo_target_text, emo_input_emo, emo_target_emo
emo_sid_df['input_text'], emo_sid_df['target_text'], emo_sid_df['input_emo'], emo_sid_df['target_emo'] = emo_sid_input_text, emo_sid_target_text, emo_sid_input_emo, emo_sid_target_emo
emo_sid_tg_df['input_text'], emo_sid_tg_df['target_text'], emo_sid_tg_df['input_emo'], emo_sid_tg_df['target_emo'] = emo_sid_tg_input_text, emo_sid_tg_target_text, emo_sid_tg_input_emo, emo_sid_tg_target_emo
emo_sid_tg_nn_df['input_text'], emo_sid_tg_nn_df['target_text'], emo_sid_tg_nn_df['input_emo'], emo_sid_tg_nn_df['target_emo'] = emo_sid_tg_nn_input_text, emo_sid_tg_nn_target_text, emo_sid_tg_nn_input_emo, emo_sid_tg_nn_target_emo
emo_sid_tg_ge_df['input_text'], emo_sid_tg_ge_df['target_text'], emo_sid_tg_ge_df['input_emo'], emo_sid_tg_ge_df['target_emo'] = emo_sid_tg_ge_input_text, emo_sid_tg_ge_target_text, emo_sid_tg_ge_input_emo, emo_sid_tg_ge_target_emo
emo_sid_tg_nn_ge_df['input_text'], emo_sid_tg_nn_ge_df['target_text'], emo_sid_tg_nn_ge_df['input_emo'], emo_sid_tg_nn_ge_df['target_emo'] = emo_sid_tg_nn_ge_input_text, emo_sid_tg_nn_ge_target_text, emo_sid_tg_nn_ge_input_emo, emo_sid_tg_nn_ge_target_emo

path = f'emotion-labeled-data/{dataset_name}'
if not os.path.exists(path):
    os.mkdir(path)

sid_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-sid-training.tsv', encoding='utf-8', sep='\t')
sid_rg_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-sid_rg-training.tsv', encoding='utf-8', sep='\t')
emo_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo-training.tsv', encoding='utf-8', sep='\t')
emo_sid_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid-training.tsv', encoding='utf-8', sep='\t')
emo_sid_tg_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid_tg-training.tsv', encoding='utf-8', sep='\t')
emo_sid_tg_nn_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid_tg_nn-training.tsv', encoding='utf-8', sep='\t')
emo_sid_tg_ge_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid_tg_ge-training.tsv', encoding='utf-8', sep='\t')
emo_sid_tg_nn_ge_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid_tg_nn_ge-training.tsv', encoding='utf-8', sep='\t')

In [ ]:
sid_df = pd.DataFrame()
sid_rg_df = pd.DataFrame()
emo_df = pd.DataFrame()
emo_sid_df = pd.DataFrame()
emo_sid_tg_df = pd.DataFrame()
emo_sid_tg_nn_df = pd.DataFrame()
emo_sid_tg_ge_df = pd.DataFrame()
emo_sid_tg_nn_ge_df = pd.DataFrame()

sid_input_text, sid_target_text, sid_input_emo, sid_target_emo = [], [], [], []
sid_rg_input_text, sid_rg_target_text, sid_rg_input_emo, sid_rg_target_emo = [], [], [], []
emo_input_text, emo_target_text, emo_input_emo, emo_target_emo = [], [], [], []
emo_sid_input_text, emo_sid_target_text, emo_sid_input_emo, emo_sid_target_emo = [], [], [], []
emo_sid_tg_input_text, emo_sid_tg_target_text, emo_sid_tg_input_emo, emo_sid_tg_target_emo = [], [], [], []
emo_sid_tg_nn_input_text, emo_sid_tg_nn_target_text, emo_sid_tg_nn_input_emo, emo_sid_tg_nn_target_emo = [], [], [], []
emo_sid_tg_ge_input_text, emo_sid_tg_ge_target_text, emo_sid_tg_ge_input_emo, emo_sid_tg_ge_target_emo = [], [], [], []
emo_sid_tg_nn_ge_input_text, emo_sid_tg_nn_ge_target_text, emo_sid_tg_nn_ge_input_emo, emo_sid_tg_nn_ge_target_emo = [], [], [], []


for index, row in eval_df.iterrows():
    if row.input_sid <= high_neg_threshold:
        input_sid_range = 'high_neg'
    elif row.input_sid > high_neg_threshold and row.input_sid < 0:
        input_sid_range = 'low_neg'
    elif row.input_sid == 0:
        input_sid_range = 'neutral'
    elif row.input_sid < high_pos_threshold and row.input_sid > 0:
        input_sid_range = 'low_pos'
    elif row.input_sid >= high_pos_threshold:
        input_sid_range = 'high_pos'
        
    if row.target_sid <= high_neg_threshold:
        target_sid_range = 'high_neg'
    elif row.target_sid > high_neg_threshold and row.input_sid < 0:
        target_sid_range = 'low_neg'
    elif row.target_sid == 0:
        target_sid_range = 'neutral'
    elif row.target_sid < high_pos_threshold and row.input_sid > 0:
        target_sid_range = 'low_pos'
    elif row.target_sid >= high_pos_threshold:
        target_sid_range = 'high_pos'
    
    if row.input_emo in high_neg_emo:
        input_emo_range = 'high_neg'
    elif row.input_emo in low_neg_emo:
        input_emo_range = 'low_neg'
    elif row.input_emo in neutral_emo:
        input_emo_range = 'neutral'
    elif row.input_emo in low_pos_emo:
        input_emo_range = 'low_pos'
    elif row.input_emo in high_pos_emo:
        input_emo_range = 'high_pos'
        
    if row.target_emo in high_neg_emo:
        target_emo_range = 'high_neg'
    elif row.target_emo in low_neg_emo:
        target_emo_range = 'low_neg'
    elif row.target_emo in neutral_emo:
        target_emo_range = 'neutral'
    elif row.target_emo in low_pos_emo:
        target_emo_range = 'low_pos'
    elif row.target_emo in high_pos_emo:
        target_emo_range = 'high_pos'
        
    if (row.input_sid < 0 and row.target_sid <= 0) & (row.input_sid < row.target_sid):
        sid_input_text.append(row.input_text)
        sid_target_text.append(row.target_text)
        sid_input_emo.append(row.input_sid)
        sid_target_emo.append(row.target_sid)
    
    elif (row.input_sid <= 0 and row.target_sid < 0) & (row.input_sid > row.target_sid):
        sid_input_text.append(row.target_text)
        sid_target_text.append(row.input_text)
        sid_input_emo.append(row.target_sid)
        sid_target_emo.append(row.input_sid)
    
    elif (row.input_sid > 0 and row.target_sid >= 0) & (row.input_sid > row.target_sid):
        sid_input_text.append(row.input_text)
        sid_target_text.append(row.target_text)
        sid_input_emo.append(row.input_sid)
        sid_target_emo.append(row.target_sid)
    
    elif (row.input_sid >= 0 and row.target_sid > 0) & (row.input_sid < row.target_sid):
        sid_input_text.append(row.target_text)
        sid_target_text.append(row.input_text)
        sid_input_emo.append(row.target_sid)
        sid_target_emo.append(row.input_sid)
        
        
    if ((input_sid_range == 'high_neg') and (target_sid_range == 'low_neg' or target_sid_range == 'neutral')) | \
       (input_sid_range == 'low_neg' and target_sid_range == 'neutral'):
        sid_rg_input_text.append(row.input_text)
        sid_rg_target_text.append(row.target_text)
        sid_rg_input_emo.append(input_sid_range)
        sid_rg_target_emo.append(target_sid_range)
    
    elif ((target_sid_range == 'high_neg') and (input_sid_range == 'low_neg' or input_sid_range == 'neutral')) | \
       (target_sid_range == 'low_neg' and input_sid_range == 'neutral'):
        sid_rg_input_text.append(row.target_text)
        sid_rg_target_text.append(row.input_text)
        sid_rg_input_emo.append(target_sid_range)
        sid_rg_target_emo.append(input_sid_range)
    
    elif ((input_sid_range == 'high_pos') and (target_sid_range == 'low_pos' or target_sid_range == 'neutral')) | \
       (input_sid_range == 'low_pos' and target_sid_range == 'neutral'):
        sid_rg_input_text.append(row.input_text)
        sid_rg_target_text.append(row.target_text)
        sid_rg_input_emo.append(input_sid_range)
        sid_rg_target_emo.append(target_sid_range)
    
    elif ((target_sid_range == 'high_pos') and (input_sid_range == 'pos' or input_sid_range == 'neutral')) | \
       (target_sid_range == 'low_pos' and input_sid_range == 'neutral'):
        sid_rg_input_text.append(row.target_text)
        sid_rg_target_text.append(row.input_text)
        sid_rg_input_emo.append(target_sid_range)
        sid_rg_target_emo.append(input_sid_range)
        
        
    if ((row.input_emo in high_neg_emo) & (row.target_emo in low_neg_emo.union(neutral_emo))) | \
       ((row.input_emo in low_neg_emo) & (row.target_emo in neutral_emo)): 
        emo_input_text.append(row.input_text)
        emo_target_text.append(row.target_text)
        emo_input_emo.append(input_emo_range)
        emo_target_emo.append(target_emo_range)
        
        if (row.input_sid < row.target_sid):
            emo_sid_input_text.append(row.input_text)
            emo_sid_target_text.append(row.target_text)
            emo_sid_input_emo.append(input_emo_range)
            emo_sid_target_emo.append(target_emo_range)
            
            if row.input_emo in emotion_transitions.keys() and row.target_emo in emotion_transitions[row.input_emo]:
                emo_sid_tg_input_text.append(row.input_text)
                emo_sid_tg_target_text.append(row.target_text)
                emo_sid_tg_input_emo.append(input_emo_range)
                emo_sid_tg_target_emo.append(target_emo_range)
                
                emo_sid_tg_ge_input_text.append(row.input_text)
                emo_sid_tg_ge_target_text.append(row.target_text)
                emo_sid_tg_ge_input_emo.append(row.input_emo)
                emo_sid_tg_ge_target_emo.append(row.target_emo)
                
                if ((row.input_emo != 'neutral') & (row.target_emo != 'neutral')):
                    emo_sid_tg_nn_input_text.append(row.input_text)
                    emo_sid_tg_nn_target_text.append(row.target_text)
                    emo_sid_tg_nn_input_emo.append(input_emo_range)
                    emo_sid_tg_nn_target_emo.append(target_emo_range)
                    
                    emo_sid_tg_nn_ge_input_text.append(row.input_text)
                    emo_sid_tg_nn_ge_target_text.append(row.target_text)
                    emo_sid_tg_nn_ge_input_emo.append(row.input_emo)
                    emo_sid_tg_nn_ge_target_emo.append(row.target_emo)

    elif ((row.input_emo in neutral_emo) & (row.target_emo in low_neg_emo.union(high_neg_emo))) | \
       ((row.input_emo in low_neg_emo) & (row.target_emo in high_neg_emo)):  
        emo_input_text.append(row.target_text)
        emo_target_text.append(row.input_text)
        emo_input_emo.append(target_emo_range)
        emo_target_emo.append(input_emo_range)
        
        if (row.input_sid > row.target_sid):
            emo_sid_input_text.append(row.target_text)
            emo_sid_target_text.append(row.input_text)
            emo_sid_input_emo.append(target_emo_range)
            emo_sid_target_emo.append(input_emo_range) 
            
            if row.input_emo in emotion_transitions.keys() and row.target_emo in emotion_transitions[row.input_emo]:
                emo_sid_tg_input_text.append(row.target_text)
                emo_sid_tg_target_text.append(row.input_text)
                emo_sid_tg_input_emo.append(target_emo_range)
                emo_sid_tg_target_emo.append(input_emo_range) 
                
                emo_sid_tg_ge_input_text.append(row.target_text)
                emo_sid_tg_ge_target_text.append(row.input_text)
                emo_sid_tg_ge_input_emo.append(row.target_emo)
                emo_sid_tg_ge_target_emo.append(row.input_emo)
                
                if ((row.input_emo != 'neutral') & (row.target_emo != 'neutral')):
                    emo_sid_tg_nn_input_text.append(row.target_text)
                    emo_sid_tg_nn_target_text.append(row.input_text)
                    emo_sid_tg_nn_input_emo.append(target_emo_range)
                    emo_sid_tg_nn_target_emo.append(input_emo_range) 
                    
                    emo_sid_tg_nn_ge_input_text.append(row.target_text)
                    emo_sid_tg_nn_ge_target_text.append(row.input_text)
                    emo_sid_tg_nn_ge_input_emo.append(row.target_emo)
                    emo_sid_tg_nn_ge_target_emo.append(row.input_emo)
    
    elif ((row.input_emo in high_pos_emo) & (row.target_emo in low_pos_emo.union(neutral_emo))) | \
       ((row.input_emo in low_pos_emo) & (row.target_emo in neutral_emo)): 
        emo_input_text.append(row.input_text)
        emo_target_text.append(row.target_text)
        emo_input_emo.append(input_emo_range)
        emo_target_emo.append(target_emo_range)
        
        if (row.input_sid > row.target_sid):
            emo_sid_input_text.append(row.input_text)
            emo_sid_target_text.append(row.target_text)
            emo_sid_input_emo.append(input_emo_range)
            emo_sid_target_emo.append(target_emo_range)
            
            if row.input_emo in emotion_transitions.keys() and row.target_emo in emotion_transitions[row.input_emo]:
                emo_sid_tg_input_text.append(row.input_text)
                emo_sid_tg_target_text.append(row.target_text)
                emo_sid_tg_input_emo.append(input_emo_range)
                emo_sid_tg_target_emo.append(target_emo_range)
                
                emo_sid_tg_ge_input_text.append(row.input_text)
                emo_sid_tg_ge_target_text.append(row.target_text)
                emo_sid_tg_ge_input_emo.append(row.input_emo)
                emo_sid_tg_ge_target_emo.append(row.target_emo)
                
                if ((row.input_emo != 'neutral') & (row.target_emo != 'neutral')):
                    emo_sid_tg_nn_input_text.append(row.input_text)
                    emo_sid_tg_nn_target_text.append(row.target_text)
                    emo_sid_tg_nn_input_emo.append(input_emo_range)
                    emo_sid_tg_nn_target_emo.append(target_emo_range)
                    
                    emo_sid_tg_nn_ge_input_text.append(row.input_text)
                    emo_sid_tg_nn_ge_target_text.append(row.target_text)
                    emo_sid_tg_nn_ge_input_emo.append(row.input_emo)
                    emo_sid_tg_nn_ge_target_emo.append(row.target_emo)
                    
    elif ((row.input_emo in neutral_emo) & (row.target_emo in low_pos_emo.union(high_pos_emo))) | \
       ((row.input_emo in low_pos_emo) & (row.target_emo in high_pos_emo)): 
        emo_input_text.append(row.target_text)
        emo_target_text.append(row.input_text)
        emo_input_emo.append(target_emo_range)
        emo_target_emo.append(input_emo_range)
        
        if (row.input_sid < row.target_sid):
            emo_sid_input_text.append(row.target_text)
            emo_sid_target_text.append(row.input_text)
            emo_sid_input_emo.append(target_emo_range)
            emo_sid_target_emo.append(input_emo_range) 
            
            if row.input_emo in emotion_transitions.keys() and row.target_emo in emotion_transitions[row.input_emo]:
                emo_sid_tg_input_text.append(row.target_text)
                emo_sid_tg_target_text.append(row.input_text)
                emo_sid_tg_input_emo.append(target_emo_range)
                emo_sid_tg_target_emo.append(input_emo_range) 
                
                if ((row.input_emo != 'neutral') & (row.target_emo != 'neutral')):
                    emo_sid_tg_nn_input_text.append(row.target_text)
                    emo_sid_tg_nn_target_text.append(row.input_text)
                    emo_sid_tg_nn_input_emo.append(target_emo_range)
                    emo_sid_tg_nn_target_emo.append(input_emo_range) 
                    
sid_df['input_text'], sid_df['target_text'], sid_df['input_emo'], sid_df['target_emo'] = sid_input_text, sid_target_text, sid_input_emo, sid_target_emo
sid_rg_df['input_text'], sid_rg_df['target_text'], sid_rg_df['input_emo'], sid_rg_df['target_emo'] = sid_rg_input_text, sid_rg_target_text, sid_rg_input_emo, sid_rg_target_emo
emo_df['input_text'], emo_df['target_text'], emo_df['input_emo'], emo_df['target_emo'] = emo_input_text, emo_target_text, emo_input_emo, emo_target_emo
emo_sid_df['input_text'], emo_sid_df['target_text'], emo_sid_df['input_emo'], emo_sid_df['target_emo'] = emo_sid_input_text, emo_sid_target_text, emo_sid_input_emo, emo_sid_target_emo
emo_sid_tg_df['input_text'], emo_sid_tg_df['target_text'], emo_sid_tg_df['input_emo'], emo_sid_tg_df['target_emo'] = emo_sid_tg_input_text, emo_sid_tg_target_text, emo_sid_tg_input_emo, emo_sid_tg_target_emo
emo_sid_tg_nn_df['input_text'], emo_sid_tg_nn_df['target_text'], emo_sid_tg_nn_df['input_emo'], emo_sid_tg_nn_df['target_emo'] = emo_sid_tg_nn_input_text, emo_sid_tg_nn_target_text, emo_sid_tg_nn_input_emo, emo_sid_tg_nn_target_emo
emo_sid_tg_ge_df['input_text'], emo_sid_tg_ge_df['target_text'], emo_sid_tg_ge_df['input_emo'], emo_sid_tg_ge_df['target_emo'] = emo_sid_tg_ge_input_text, emo_sid_tg_ge_target_text, emo_sid_tg_ge_input_emo, emo_sid_tg_ge_target_emo
emo_sid_tg_nn_ge_df['input_text'], emo_sid_tg_nn_ge_df['target_text'], emo_sid_tg_nn_ge_df['input_emo'], emo_sid_tg_nn_ge_df['target_emo'] = emo_sid_tg_nn_ge_input_text, emo_sid_tg_nn_ge_target_text, emo_sid_tg_nn_ge_input_emo, emo_sid_tg_nn_ge_target_emo

path = f'emotion-labeled-data/{dataset_name}'
if not os.path.exists(path):
    os.mkdir(path)

sid_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-sid-testing.tsv', encoding='utf-8', sep='\t')
sid_rg_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-sid_rg-testing.tsv', encoding='utf-8', sep='\t')
emo_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo-testing.tsv', encoding='utf-8', sep='\t')
emo_sid_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid-testing.tsv', encoding='utf-8', sep='\t')
emo_sid_tg_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid_tg-testing.tsv', encoding='utf-8', sep='\t')
emo_sid_tg_nn_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid_tg_nn-testing.tsv', encoding='utf-8', sep='\t')
emo_sid_tg_ge_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid_tg_ge-testing.tsv', encoding='utf-8', sep='\t')
emo_sid_tg_nn_ge_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid_tg_nn_ge-testing.tsv', encoding='utf-8', sep='\t')

In [ ]:
sid_train_df = pd.read_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-sid-training.tsv', encoding='utf-8', sep='\t')
sid_test_df = pd.read_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-sid-testing.tsv', encoding='utf-8', sep='\t')

sid_rg_train_df = pd.read_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-sid_rg-training.tsv', encoding='utf-8', sep='\t')
sid_rg_test_df = pd.read_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-sid_rg-testing.tsv', encoding='utf-8', sep='\t')

emo_train_df = pd.read_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo-training.tsv', encoding='utf-8', sep='\t')
emo_test_df = pd.read_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo-testing.tsv', encoding='utf-8', sep='\t')

emo_sid_train_df = pd.read_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid-training.tsv', encoding='utf-8', sep='\t')
emo_sid_test_df = pd.read_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid-testing.tsv', encoding='utf-8', sep='\t')

emo_sid_tg_train_df = pd.read_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid_tg-training.tsv', encoding='utf-8', sep='\t')
emo_sid_tg_test_df = pd.read_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid_tg-testing.tsv', encoding='utf-8', sep='\t')

emo_sid_tg_nn_train_df = pd.read_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid_tg_nn-training.tsv', encoding='utf-8', sep='\t')
emo_sid_tg_nn_test_df = pd.read_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid_tg_nn-testing.tsv', encoding='utf-8', sep='\t')

emo_sid_tg_ge_train_df = pd.read_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid_tg_ge-training.tsv', encoding='utf-8', sep='\t')
emo_sid_tg_ge_test_df = pd.read_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid_tg_ge-testing.tsv', encoding='utf-8', sep='\t')

emo_sid_tg_nn_ge_train_df = pd.read_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid_tg_nn_ge-training.tsv', encoding='utf-8', sep='\t')
emo_sid_tg_nn_ge_test_df = pd.read_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid_tg_nn_ge-testing.tsv', encoding='utf-8', sep='\t')

def gen_prefix(df):
    prefixes = []
    for index, row in df.iterrows():
        input_emotion = row.input_emo
        target_emotion = row.target_emo
        prefix =  str(input_emotion) + " to " + str(target_emotion)
        prefix = prefixes.append(prefix)
    df.insert(0, "prefix", prefixes)
    df = df.drop(['Unnamed: 0'], axis=1)
    return df

sid_train_df = gen_prefix(sid_train_df)
sid_test_df = gen_prefix(sid_test_df)

sid_rg_train_df = gen_prefix(sid_rg_train_df)
sid_rg_test_df = gen_prefix(sid_rg_test_df)

emo_train_df = gen_prefix(emo_train_df)
emo_test_df = gen_prefix(emo_test_df)

emo_sid_train_df = gen_prefix(emo_sid_train_df)
emo_sid_test_df = gen_prefix(emo_sid_test_df)

emo_sid_tg_train_df = gen_prefix(emo_sid_tg_train_df)
emo_sid_tg_test_df = gen_prefix(emo_sid_tg_test_df)

emo_sid_tg_nn_train_df = gen_prefix(emo_sid_tg_nn_train_df)
emo_sid_tg_nn_test_df = gen_prefix(emo_sid_tg_nn_test_df)

emo_sid_tg_ge_train_df = gen_prefix(emo_sid_tg_ge_train_df)
emo_sid_tg_ge_test_df = gen_prefix(emo_sid_tg_ge_test_df)

emo_sid_tg_nn_ge_train_df = gen_prefix(emo_sid_tg_nn_ge_train_df)
emo_sid_tg_nn_ge_test_df = gen_prefix(emo_sid_tg_nn_ge_test_df)

sid_train_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-sid-training_t5.tsv', encoding='utf-8', sep='\t')
sid_test_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-sid-testing_t5.tsv', encoding='utf-8', sep='\t')

sid_rg_train_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-sid_rg-training_t5.tsv', encoding='utf-8', sep='\t')
sid_rg_test_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-sid_rg-testing_t5.tsv', encoding='utf-8', sep='\t')

emo_train_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo-training_t5.tsv', encoding='utf-8', sep='\t')
emo_test_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo-testing_t5.tsv', encoding='utf-8', sep='\t')

emo_sid_train_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid-training_t5.tsv', encoding='utf-8', sep='\t')
emo_sid_test_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid-testing_t5.tsv', encoding='utf-8', sep='\t')

emo_sid_tg_train_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid_tg-training_t5.tsv', encoding='utf-8', sep='\t')
emo_sid_tg_test_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid_tg-testing_t5.tsv', encoding='utf-8', sep='\t')

emo_sid_tg_nn_train_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid_tg_nn-training_t5.tsv', encoding='utf-8', sep='\t')
emo_sid_tg_nn_test_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid_tg_nn-testing_t5.tsv', encoding='utf-8', sep='\t')

emo_sid_tg_ge_train_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid_tg_ge-training_t5.tsv', encoding='utf-8', sep='\t')
emo_sid_tg_ge_test_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid_tg_ge-testing_t5.tsv', encoding='utf-8', sep='\t')

emo_sid_tg_nn_ge_train_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid_tg_nn_ge-training_t5.tsv', encoding='utf-8', sep='\t')
emo_sid_tg_nn_ge_test_df.to_csv(f'emotion-labeled-data/{dataset_name}/{dataset_name}-emo_sid_tg_nn_ge-testing_t5.tsv', encoding='utf-8', sep='\t')